# Build English Verb Dataset

In this notebook, we build a translation dataset based on NIV and ESV translation
alignments provided by GBI. The GBI data, which uses an underlying WLC Hebrew text
has already been aligned to the Amsterdam BHSA Hebrew dataset in 
[GBI_alignment_wrangling.ipynb](GBI_alignment_wrangling.ipynb). We can thus take
advantage of both databases and their associated data when building our dataset here.

In the dataset we'll attempt to parse the English text so that the syntax and (especially)
the verbal forms can be analyzed alongside the Hebrew grammar. We'll start out with 
Spacy for the English parsings. 

In [159]:
import re
import json
import collections
import re
import pandas as pd
pd.set_option('display.max_rows', 200)
from pathlib import Path
from tf.app import use
from bidict import bidict # bidirectional dictionary

# custom modules 
import tf_tools
from gbi_functions import id2ref
from positions import PositionsTF

# organize pathways
PROJ_DIR = Path.home().joinpath('github/CambridgeSemiticsLab/translation_traditions_HB')
PRIVATE_DATA = PROJ_DIR.joinpath('data/_private_')
GBI_DATA_DIR = PRIVATE_DATA.joinpath('GBI_alignment')
VERB_DIR = PRIVATE_DATA.joinpath('verb_data')

# load GBI data
gbi_niv = json.loads(GBI_DATA_DIR.joinpath('niv84.ot.alignment.json').read_text())
gbi_esv = json.loads(GBI_DATA_DIR.joinpath('esv.ot.alignment.json').read_text())

# load BHSA / GBI Alignment
bhsa2gbi = json.loads(GBI_DATA_DIR.joinpath('bhsa2gbi.json').read_text())

# load BHSA data and methods
bhsa = use('bhsa')
api = bhsa.api
F, E, T, L = api.F, api.E, api.T, api.L

In [2]:
# set up some dictionaries for convenient word data access
# see 'Dict demos' in next cells for quick intro to the resulting dict structures

sources = (('niv', gbi_niv), ('esv', gbi_esv))
word_data = collections.defaultdict(lambda: collections.defaultdict(list))
verse2words = collections.defaultdict(lambda: collections.defaultdict(list)) 
linkbyid = collections.defaultdict(list) # list of 2-tuples, each containing word IDs
id2link = collections.defaultdict(dict) # select a link based on a single ID 

for name, source in sources:
    for verse in source:
        
        # unpack words for processing
        trans_words =  verse['translation']['words']
        manu_words = verse['manuscript']['words']
        
        # map translation word data
        for w in trans_words:
            ref_tuple = id2ref(w['id'], 'translation')
            verse2words[name][ref_tuple].append(w['id'])
            word_data[name][w['id']] = w
        
        # map WLC word data
        # arbitrarily use the copy stored under NIV
        if name == 'niv':
            for w in manu_words:
                ref_tuple = id2ref(w['id'])
                verse2words['wlc'][ref_tuple].append(w['id'])
                word_data['wlc'][w['id']] = w
                
        # map links to word ids
        # the alignment data just contains indices pointing
        # to the various lists, so these have to be used to 
        # identify the specific word in question
        for wlc_indices, trans_indices in verse['links']:
            wlc_ids = tuple(manu_words[i]['id'] for i in wlc_indices)
            trans_ids = tuple(sorted(trans_words[i]['id'] for i in trans_indices))
            linkbyid[name].append((wlc_ids, trans_ids))
            for wid in wlc_ids:
                id2link[name][wid] = trans_ids

### Dict demos

In [3]:
word_data['wlc'][10010010021]

{'id': 10010010021,
 'altId': 'בָּרָ֣א\u200e-1',
 'text': 'בָּרָ֣א\u200e',
 'strong': 'H1254',
 'gloss': 'he created',
 'gloss2': '创造',
 'lemma': 'ברא_1',
 'pos': 'verb',
 'morph': 'vqp3ms'}

In [4]:
word_data['niv'][1001001005]

{'id': 1001001005,
 'altId': 'created-1',
 'text': 'created',
 'transType': 'k',
 'isPunc': False,
 'isPrimary': True}

In [5]:
verse2words['niv'][('Genesis', 1, 1)]

[1001001001,
 1001001002,
 1001001003,
 1001001004,
 1001001005,
 1001001006,
 1001001007,
 1001001008,
 1001001009,
 1001001010,
 1001001011]

In [6]:
for wlc_ids, trans_ids in linkbyid['niv']:
    if 10010010021 in wlc_ids:
        print(wlc_ids, trans_ids)

(10010010021,) (1001001005,)


In [7]:
# NB: similar to above, however
# the link is only 1-to-X
# so some parts of the left side of the link could be missing
id2link['niv'][10010010021]

(1001001005,)

## Sub-sample verbs

The basis of the dataset is verbs. The jumping-off point is the BHSA syntax data. Thus what
we do is assemble the dataset by the BHSA verbs.

First we get a one-to-one verb mapping between BHSA and GBI Hebrew (WLC). We can use the GBI hebrew
links to select the correct words in the translations.

The main stipulation for the selection is agreement between BHSA and WLC on 
the classification of a word as a verb (e.g. טוֹב); for BHSA classification, 
we use a contextual definition (phrase-dependent) which classifies whether 
the word is behaving as a verb in context (e.g. participles)

Since some alignments between BHSA and WLC are `many-to-N` or `N-to-many`, 
we also filter out any of these non-verbal words. This leaves us with
a 1-to-1 alignment, so that 1 verb in BHSA equals 1 verb in WLC.

In [8]:
# track where BHSA and WLC disagree on the classification of a verb
no_match = {
    'disagree': [],
}

verb_bhsa2gbi = {}

# find 1-to-1 matches of BHSA and WLC verbs
for bhsa_nodes, gbi_ids in bhsa2gbi:
    
    # filter out non-verbs from the links
    bhsa_verbs = [w for w in bhsa_nodes if F.pdp.v(w) == 'verb'] 
    wlc_verbs = [w for w in gbi_ids if word_data['wlc'][w]['pos'] == 'verb']
    data = (T.text(bhsa_nodes), T.sectionFromNode(bhsa_nodes[0]), bhsa_nodes, gbi_ids) # track null matches
    
    # one case, Jer 51:3, has a double verb mapping caused by 
    # ידרך ידרך, which BHSA maps to a single word node, and gbi 
    # keeps as 2 words; we disambig that here and keep only 
    # first gbi word
    if bhsa_verbs and bhsa_verbs[0] == 262780:
         wlc_verbs = wlc_verbs[:1]
    
    # skip non-verbal contexts
    if not bhsa_verbs + wlc_verbs:
        continue
    
    # track disagreements between 2 sources
    elif (bhsa_verbs and not wlc_verbs) or (wlc_verbs and not bhsa_verbs):
        no_match['disagree'].append(data)
    
    # store result both ways: bhsa 2 wlc, wlc 2 bhsa
    elif len(bhsa_verbs) == 1 and len(wlc_verbs) == 1:
        
        # make a subset selection of verbs
        bhsa_verb, wlc_verb = bhsa_verbs[0], wlc_verbs[0]
        verb_bhsa2gbi[bhsa_verb] = word_data['wlc'][wlc_verb]
        
    
    # or there's a problem...
    else:
        raise Exception(f'Misalignment at {data}')
        
print(sum(len(v) for v in no_match.values()), 'verbs do not match requirements')
print(len(verb_bhsa2gbi), 'selected for building dataset')

4427 verbs do not match requirements
68826 selected for building dataset


NB that the ~4.4k verbs in disagreement is because we use contextual parts of speech
from the BHSA dataset. The GBI dataset does not seem to be as sensitive to context for
pos. A large proportion of these cases are participles used as nouns rather than verbs.

In [9]:
no_match['disagree'][500:510]

[('פְּקֻדֵיהֶ֖ם ', ('Numbers', 1, 39), [70153], [40010390011, 40010390012]),
 ('יֹצֵ֥א ', ('Numbers', 1, 40), [70183], [40010400141]),
 ('פְּקֻדֵיהֶ֖ם ', ('Numbers', 1, 41), [70185], [40010410011, 40010410012]),
 ('יֹצֵ֥א ', ('Numbers', 1, 42), [70214], [40010420141]),
 ('פְּקֻדֵיהֶ֖ם ', ('Numbers', 1, 43), [70216], [40010430011, 40010430012]),
 ('פְּקֻדִ֡ים ', ('Numbers', 1, 44), [70229], [40010440022]),
 ('פְּקוּדֵ֥י ', ('Numbers', 1, 45), [70250], [40010450031]),
 ('פְּקֻדִ֔ים ', ('Numbers', 1, 46), [70271], [40010460032]),
 ('פְקֻדֵיהֶ֑ם ', ('Numbers', 2, 4), [70477], [40020040022, 40020040023]),
 ('פְקֻדָ֑יו ', ('Numbers', 2, 6), [70502], [40020060022, 40020060023])]

We export the dataset for later processing.

## Parsing English with Spacy

For understanding the basics of Spacy, see:
https://spacy.io/usage/linguistic-features

For each verb in the `select_verbs` dictionary, we retrieve its verse text in a
given translation. The translated text is parsed by Spacy, which supplies us with
a dependency tree, parts of speech, and verb tenses for the English side of things.

For Spacy tags used with the model of choice, see:
https://github.com/explosion/spacy-models/releases//tag/en_core_web_sm-2.3.1

In [10]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Doc, Token, Span
from spacy.util import filter_spans # filter overlaps; nice tip: https://stackoverflow.com/a/63303480/8351428

### Build links between BHSA and English translations via WLC

In [11]:
# build dataset

english_verbs = collections.defaultdict(dict)
not_linked = collections.defaultdict(list)

for trans in ('esv', 'niv'):
    
    for bhsa_node, wlc_word in verb_bhsa2gbi.items():
        try:
            english_verbs[trans][bhsa_node] = id2link[trans][wlc_word['id']]
        except: 
            not_linked[trans].append(wlc_word)
        
    print(f'n-verbs selected for {trans}: {len(english_verbs[trans])}')
    print(f'\tn-verbs unlinked: {len(not_linked[trans])}')

n-verbs selected for esv: 66837
	n-verbs unlinked: 1989
n-verbs selected for niv: 65455
	n-verbs unlinked: 3371


### Parse English Data within its Verse Context

In [12]:
# to avoid double-parsing verses, we map
# all cases needed to be parsed to a single
# parsed verse

# for each translation, gather the verse labels
# that need to be parsed

versestoparse = collections.defaultdict(set)
for trans in english_verbs:
    for bhsa_node, trans_words in english_verbs[trans].items():
        ref = id2ref(trans_words[0], 'translation')
        versestoparse[trans].add(ref)
        
    print(f'{trans} has {len(versestoparse[trans])} verses to parse')

esv has 21264 verses to parse
niv has 21251 verses to parse


### Apply raw Spacy parsing to all relevant verses

In [13]:
# set up the Spacy processor as well as some customized attributes
nlp = spacy.load('en_core_web_sm')
Span.set_extension('tam_tag', default='', force=True)

In [14]:
def process_tokens(gbi_words):
    """Prepare GBI pre-tokenized words for Spacy Doc construction"""
    gbi_text = [w['text'] for w in gbi_words]
    doc = Doc(nlp.vocab, words=gbi_text)
    return doc

# edit default tokenizer in order to feed in 
# already-tokenized GBI words
nlp.tokenizer = process_tokens

def parse_verse(verse_tuple, translation):
    """Parse translation verse with Spacy."""
    tokens = [word_data[translation][w] for w in verse2words[translation][verse_tuple]]
    parsed_doc = nlp(tokens, translation) # magic happens here
    return parsed_doc

def parse_verses(transdict):
    """Iterate through all verses and parse them.
    
    Args:
        versedict: dict with structure of e.g. {'niv': set(('Genesis', 1, 1)...}}
    Returns:
        dict w/ structure of e.g. {'niv': {('Genesis', 1, 1): Spacy.Doc}}
    """
    
    parsed_verses = collections.defaultdict(dict)
    
    bhsa.indent(0, reset=True)
    bhsa.info(f'Parsing translations...')
    
    for translation, verse_set in transdict.items():
    
        # it takes a long time so we time it
        bhsa.indent(1)
        bhsa.info(f'Beginning {translation}...')
        bhsa.indent(2)
    
        # parse the verse and put Spacy.Doc in a dict
        for i, ref_tuple in enumerate(verse_set):
            
            if i % 5000 == 0 and i != 0:
                bhsa.info(f'done with verse {i}')
                
            parsed_verses[translation][ref_tuple] = parse_verse(ref_tuple, translation)
            
        bhsa.indent(1)
        bhsa.info('done!')
    
    return parsed_verses

Now we execute the parser for all verses. This will take some time!

In [149]:
# toggle here to run fresh parsings
parsed_verses_file = '_private_/parsings/parsed_verses.pickle'
if False:
    parsed_verses = parse_verses(versestoparse)
    with open(parsed_verses_file, 'wb') as outfile:
        pickle.dump(parsed_verses, outfile)
else:
    with open(parsed_verses_file, 'rb') as infile:
        parsed_verses = pickle.load(infile)

In [150]:
parsed_verses['niv'][('Genesis', 1, 1)]

In the beginning God created the heavens and the earth . 

### Set up Matcher rules for advanced TAM tags

Spacy parses raw strings into tags and dependencies. For verbs we are particularly
interested in tense, aspect, and modality (TAM). The default tags are not very informative with 
regard to TAM. But we can also achieve these labels ourselves by adding some additional rules.

We will use Spacy's Matcher class for this, alongside the parser:

https://spacy.io/usage/rule-based-matching

To-do list of primary English tense constructions, curated from:

https://en.wikipedia.org/wiki/English_verbs#Expressing_tenses,_aspects_and_moods

```
simple present            writes
simple past               wrote
present progressive       is writing
past progressive          was writing
present perfect           has written
past perfect              had written
present perf. progress.   has been writing
past perf. progress.      had been writing
future                    will write
future perfect            will have written
future perf. progress.    will have been writing
```

secondary constructions:

```
imperative               write
future-in-past           would write
do-support               does write
be-going-to future       is going to write
```

Later on, we can consider dividing these constructions up into 3 columns -- 1 each for 
tense, aspect, and modality. If a construction contributes to one of these categories,
the column gets filled. Otherwise it is left empty. 

```
"has been writing"

tense           aspect            modality
-----           ------             ------
past      perfect progressive
```

In [226]:
# a set of rules to match tense-aspect-modality construtions in English
# overlapping results will be filtered out and the longest matching span
# will be kept in its place

# these patterns can be inserted between verb auxiliaries
# and their heads to represent any number of interrupting
# adverbial modifiers
advb_pronouns = {'TAG': {'IN':['RB', 'PRP']}, 'OP': '*'}
advbs = {'TAG': {'IN':['RB']}, 'OP': '*'}
    
tam_rules = [
    (
        'present', 
        [
            {'TAG':{'IN':['VBZ', 'VBP']}, 'DEP': {'NOT_IN': ['aux']}},
        ]
    ),
    (
        'present progressive', 
        [
            {'TAG': {'IN':['VBZ', 'VBP']}, 'LEMMA':'be'},
            advb_pronouns,
            {'TAG':'VBG', 'LEMMA': {'NOT_IN':['go']}},
        ]
    ),
    (
        'present perfect',
        [
            {'TAG': {'IN': ['VBZ', 'VBP']}, 'LEMMA': 'have'},
            advb_pronouns,
            {'TAG': 'VBN', 'DEP': {'NOT_IN': ['aux']}},
        ]
    ),
    (
        'present perfect progressive',
        [
            {'TAG': {'IN': ['VBZ', 'VBP']}, 'LEMMA': 'have'},
            advb_pronouns,
            {'TAG': 'VBN', 'LEMMA': 'be'},
            {'TAG': 'VBG'},
        ]
    ),
    (
        'past',
        [
            {'TAG': 'VBD', 'DEP': {'NOT_IN':['aux']}},
        ]
    ),
    (
        'past perfect',
        [
            {'TAG': {'IN': ['VBD']}, 'LEMMA': 'have'},
            advb_pronouns,
            {'TAG': 'VBN', 'DEP': {'NOT_IN': ['aux']}},
        ]
    ),
    (
        'past perfect progressive',
        [
            {'TAG': {'IN': ['VBD']}, 'LEMMA': 'have'},
            advb_pronouns,
            {'TAG': 'VBN', 'LEMMA': 'be'},
            {'TAG': 'VBG'},
        ]
    ),
    (
        'future perfect',
        [
            {'TAG': 'MD', 'LEMMA': 'will'},
            advb_pronouns,
            {'TAG': {'IN': ['VB']}, 'LEMMA': 'have'},
            advb_pronouns,
            {'TAG': 'VBN', 'DEP': {'NOT_IN': ['aux']}},
        ]
    ),
    (
        'future perfect progressive',
        [
            {'TAG': 'MD', 'LEMMA': 'will'},
            advb_pronouns,
            {'TAG': {'IN': ['VB']}, 'LEMMA': 'have'},
            advb_pronouns,
            {'TAG': 'VBN', 'LEMMA': 'be'},
            {'TAG': 'VBG'},
        ]
    ),

    (
        'past progressive',
        [
            {'TAG':'VBD', 'LEMMA':'be'},
            advb_pronouns,
            {'TAG': 'VBG'},
        ]
    ),
    (
        'future',
        [
            {'TAG': 'MD', 'LEMMA': 'will'},
            advb_pronouns,
            {'TAG': 'VB', 'DEP': {'NOT_IN': ['aux']}},
        ]
    ),
    (
        'future-in-past', # habitual?
        [
            {'LOWER': 'would', 'DEP': {'IN': ['aux']}},
            advb_pronouns,
            {'TAG':'VB'}
        ]
    ),
    (
        'do-support present',
        [
            {'TAG': {'IN': ['VBZ', 'VBP']}, 'LEMMA': 'do'},
            advb_pronouns,
            {'TAG': 'VB'},
        ]
    ),
    (
        'past perfect (did)',
        [
            {'TAG': {'IN': ['VBD']}, 'LEMMA': 'do'},
            advb_pronouns,
            {'TAG': 'VB'},
        ]
    ),
    (
        'be-going-to future',
        [
            {'TAG': {'IN':['VBZ', 'VBP']}, 'LEMMA':'be'}, 
            advb_pronouns,
            {'TAG': 'VBG', 'LEMMA': 'go'},
            {'TAG': 'TO'},
            {'TAG': 'VB'},
        ]
    ),
    (
        'modal',
        [
            {'TAG': {'IN':['VB', 'MD']}, 'lower': {'IN':['let', 'may', 'shall', 'must']}},
            {'TAG': {'NOT_IN':['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']}, 'OP': '*'},
            {'TAG': 'VB'},
        ]
    ),
    (
        'present participle', 
        [
            {'TAG': 'VBG', 'DEP': {'NOT_IN':['aux']}},
        ]
    ),
    (
        'present participle',
        [
            {'TAG': 'JJ', 'LOWER':{'REGEX':'^.+ing$'}},
        ]
    ),
    (
        'to infinitive',
        [
            {'TAG': 'TO'},
            {'TAG':'VB'}
        ]
    ),
    
    # add another modal category:
    # "Let him go up"
    # i.e. "Let/may ... verb"
  
# This pattern as-is is too over-active in matches
# need to find another way to match it
#     (
#         'imperative',
#         [
#             {'TAG': 'VB', 'DEP':{'NOT_IN':['aux']}},
#         ]
#     ),
    
]

# def on_match(matcher, doc, mid, matches):
#     for match in matches:
#         begin, end = match[1:]
#         tam_matches[(begin, end)].add(match)

matcher = Matcher(nlp.vocab)

# add all rules to the matcher object
for tag, rules in tam_rules:
    matcher.add(tag, None, rules)

In [227]:
# for every verse isolate the set of relevant spans
# which match the TAM rules and map to verses
verse2spans = collections.defaultdict(dict)
for trans, ref_tuples in parsed_verses.items():
    for ref_tuple, spacy_doc in ref_tuples.items():
        matches = matcher(spacy_doc)
        
        # retrieve Spacy Span objects
        # and give them TAM tags
        spans = []
        for m_id, start, end in matches:
            span = spacy_doc[start:end]
            span._.tam_tag = nlp.vocab.strings[m_id]
            spans.append(span)
        
        # filter out overlapping spans and keep 
        # only the longest strings
        filtered_spans = filter_spans(spans)
        
        # save positive matches; unmatched verses will
        # be recognized later
        if filtered_spans:
            verse2spans[trans][ref_tuple] = filtered_spans
        else:
            continue
            
    print(f'{trans} n-spans: {len(verse2spans[trans])}')

esv n-spans: 20870
niv n-spans: 20730


In [228]:
# # export a set for inspection
# inspect = ''
# for verse_ref, spans in verse2spans['niv'].items():
#     inspect += str(verse_ref) + '\n'
#     inspect += str(parsed_verses['niv'][verse_ref]) + '\n'
#     for span in spans:
#         inspect += f'\t{span} -> {span._.tam_tag}\n'
        
#     inspect += '\n\n'

# Path('inspect_tam.txt').write_text(inspect)

In [229]:
# for w in L.d(1414423,'word'):
#     print(w, T.text(w))

In [230]:
# for w in [w for w in verse2words['wlc'][('Genesis', 3, 11)]]:
    
#     print(w, word_data['wlc'][w]['text'])
#     print(id2link['niv'].get(w))
#     print(' '.join(word_data['niv'][w]['text'] for w in id2link['niv'].get(w, [])))
#     print()

In [242]:
def display_tokens(doc):
    for token in doc:
        print(token)
        print('  ', token.lemma_)
        print('  ', token.pos_)
        print('  ', token.tag_)
        print('  ', token.dep_)
        print('  ', token.shape_)
        
#display_tokens(parsed_verses['niv'][('Genesis', 1, 24)])

In [232]:
nlp2 = spacy.load('en_core_web_sm')

In [233]:
#display_tokens(nlp2('Let there be an expanse between the waters to separate'))

In [234]:
verse2spans['niv'][('Genesis', 1, 3)]

[said, Let there be, was]

In [235]:
spacy.explain('IN')

'conjunction, subordinating or preposition'

### Link spans to verbs

We will now attempt to re-link the spans with the verbs.

In [247]:
def trans_to_span(trans_words, spans, verse_words):
    """Match given words with its TAM span.
    
    The links are made based on set overlap.
    """
    for span in spans:
        span_words = set(verse_words[span.start:span.end])
        if set(trans_words) & span_words:
            return span

verse_inspect = collections.defaultdict(lambda: collections.defaultdict(str)) # for exporting an inspection document
bhsa2eng = collections.defaultdict(dict)

for trans, bhsa_nodes in english_verbs.items():
    
    for bhsa_node, eng_words in bhsa_nodes.items():

        inspect = '' # for debugging and inspection
        
        verse_ref = id2ref(eng_words[0], 'translation')
        eng_text = ' '.join(word_data[trans][w]['text'] for w in eng_words)

        # try to retrieve span links with advanced TAM tags
        verse_words = verse2words[trans][verse_ref]
        spans = verse2spans[trans].get(verse_ref, [])
        span_match = trans_to_span(eng_words, spans, verse_words) or ''
        if span_match:
            tam_tag = span_match._.tam_tag
        else:
            tam_tag = ''
        
        # retrieve basic parsings
        raw_tokens = []
        for i, token in enumerate(parsed_verses[trans][verse_ref]):
            if verse_words[i] in eng_words:
                raw_tokens.append(token)
                
        vb_tokens = [t for t in raw_tokens if t.tag_.startswith('VB')]
            
        # save the data
        data = {
            'words': eng_text,
            'tags': '|'.join(t.tag_ for t in raw_tokens),
            'vb_tags': '|'.join(t.tag_ for t in vb_tokens),
            'TAM_cx': tam_tag,
            'TAM_span': f'{span_match}',
        }
        
        bhsa2eng[trans][bhsa_node] = data
            
        # add strings to inspection file
        if span_match:
            verse_inspect[trans][verse_ref] += f'\tMATCH: {eng_text}\n'
            verse_inspect[trans][verse_ref] += f'\t\t{span_match} -> {span_match._.tam_tag}\n'
        else:
            verse_inspect[trans][verse_ref] += f'\tMISS: {eng_text}\n'
            verse_inspect[trans][verse_ref] += f'\t\t{eng_text}\n'

In [251]:
# export
for trans, trans_data  in bhsa2eng.items():
    trans_file = VERB_DIR.joinpath(f'bhsa2{trans}.json')
    with open(trans_file, 'w') as outfile:
        json.dump(trans_data, outfile, ensure_ascii=False, indent=2)
    
    # export inspection file
    write = ''
    inspect_file = PRIVATE_DATA.joinpath(f'debugging/{trans}_inspect.txt')
    for verse, message in verse_inspect[trans].items():
        write += '{} {}:{}'.format(*verse) + '\n'
        write += str(parsed_verses[trans][verse]) + '\n'
        write += message
        write += '\n'
    inspect_file.write_text(write)

In [238]:
# dump the bhsa2wlc dataset
wlc_verbdataset_path = VERB_DIR.joinpath('bhsa2wlc.json') 
with open(wlc_verbdataset_path, 'w') as outfile:
    json.dump(verb_bhsa2gbi, outfile, ensure_ascii=False, indent=2)

<hr>

# Export

In [21]:
# # save these resulting dicts for later convenient use

# save_files = [
#     ('word_data', word_data),
#     ('verse2words', {k:{tuple(k2):v2 for k2,v2 in v.items()} for k,v in verse2words.items()}),
#     ('id_links', linkbyid),
# ]

# for filename, data in save_files:
#     filepath = GBI_DATA_DIR.joinpath(filename+'.json')
#     with open(filepath, 'w') as outfile:
#         json.dump(data, outfile, ensure_ascii=False, indent=2)

In [243]:
# # export the dataset
# dataset_path = PROJ_DIR.joinpath('data/_private_/translation_dataset.csv')

# data_df.to_csv(dataset_path, index=False)

<hr> 
Scratch code

In [154]:
# write = ''
# for verse, cases in verse_sees.items():
#     parsed_verse = parsed_verses['niv'][verse]
#     ref_str = '{} {}:{}'.format(*verse)
#     write += ref_str + '\n'
#     write += str(parsed_verse) + '\n'
#     for case in cases:
#         write += case
#     write += '\n'
    
# Path('see_cases.txt').write_text(write)